In [1]:
import pandas as pd
import numpy  as np
import re

In [2]:
data = pd.read_csv('HM_data_raw.csv')

In [3]:
data.head()

,product_id,product_category,product_name,product_price,style_id,color_id,product_colors,Fit,Composition,Description,Imported,scrapy_datetime
0,985159001,men_jeans_skinny,Skinny Jeans,$ 19.99,985159,1,Denim blue,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Black,Yes,2022-03-02 22:41:14.687600
1,985159001,men_jeans_skinny,Skinny Jeans,$ 19.99,985159,1,Denim blue,Skinny fit,Pocket lining: Cotton 100%,Solid-color,Yes,2022-03-02 22:41:14.687600
2,985159001,men_jeans_skinny,Skinny Jeans,$ 19.99,985159,1,Denim blue,Skinny fit,Pocket lining: Cotton 100%,Solid-color,Yes,2022-03-02 22:41:14.687600
3,985159001,men_jeans_skinny,Skinny Jeans,$ 19.99,985159,1,Denim blue,Skinny fit,Pocket lining: Cotton 100%,Solid-color,Yes,2022-03-02 22:41:14.687600
4,985159001,men_jeans_skinny,Skinny Jeans,$ 19.99,985159,1,Denim blue,Skinny fit,Pocket lining: Cotton 100%,Solid-color,Yes,2022-03-02 22:41:14.687600


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8361 entries, 0 to 8360
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        8361 non-null   int64 
 1   product_category  8361 non-null   object
 2   product_name      8361 non-null   object
 3   product_price     8361 non-null   object
 4   style_id          8361 non-null   int64 
 5   color_id          8361 non-null   int64 
 6   product_colors    8361 non-null   object
 7   Fit               8361 non-null   object
 8   Composition       8361 non-null   object
 9   Description       8361 non-null   object
 10  Imported          8361 non-null   object
 11  scrapy_datetime   8361 non-null   object
dtypes: int64(3), object(9)
memory usage: 784.0+ KB


### Formatting data

In [5]:
# column names
data = data.rename(columns = {'Fit': 'fit', 'Composition': 'composition','Description':'description', 
                              'Imported': 'imported'})

# product_name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product_price
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$', '')).astype(float)

# product_color
data['product_colors'] = data['product_colors'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower())

# fit
data['fit'] = data['fit'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

# description
data['description'] = data['description'].apply(lambda x: x.replace(' ', '_').lower())

# scrapy_datetime
data['scrapy_datetime'] = data['scrapy_datetime'].apply(lambda x: re.search('(\d{4}-\d{2}-\d{2}\s\d{2}\:\d{2}\:\d{2})', x).group(0))
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')

# column 'composition' has several information 

# For the first cycle of CRISP, let's remove "Pocket lining", "Lining", and "Shell"
data = data[~data['composition'].str.contains('Pocket lining:', na=False)]
data = data[~data['composition'].str.contains('Pocket:', na=False)]
data = data[~data['composition'].str.contains('Lining:', na=False)]
data = data[~data['composition'].str.contains('Shell:', na=False)]

In [6]:
data.index = np.arange(0, len(data))

data.head()

,product_id,product_category,product_name,product_price,style_id,color_id,product_colors,fit,composition,description,imported,scrapy_datetime
0,690449051,men_jeans_ripped,skinny_jeans,39.99,690449,51,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Spandex 2%",gray,Yes,2022-03-02 22:41:14
1,690449051,men_jeans_ripped,skinny_jeans,39.99,690449,51,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Spandex 2%",solid-color,Yes,2022-03-02 22:41:14
2,690449051,men_jeans_ripped,skinny_jeans,39.99,690449,51,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Spandex 2%",solid-color,Yes,2022-03-02 22:41:14
3,690449051,men_jeans_ripped,skinny_jeans,39.99,690449,51,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Spandex 2%",solid-color,Yes,2022-03-02 22:41:14
4,690449051,men_jeans_ripped,skinny_jeans,39.99,690449,51,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Spandex 2%",solid-color,Yes,2022-03-02 22:41:14


In [7]:
# break the actual column 'composition' by comma into a new dataframe, df1
df1 = data['composition'].str.split(',', expand=True)

# create a new empty dataframe with the columns from 'composition': cotton | polyester | elastane | spandex
# same len as data
df_ref = pd.DataFrame(index=np.arange(len(data)), columns=['cotton', 'polyester', 'spandex'])

# attach df_ref to df1 column by column:

# column df1[0]
df1[0].unique() # check the unique values; in this case it's all cotton

# cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'
df_ref = pd.concat([df_ref, df_cotton], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# column df1[1]
df1[1].unique()  # it shows we have 'polyester' and 'spandex'

# polyester from column df1[1]
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'
df_ref = pd.concat([df_ref, df_polyester], axis=1) # overwrites the rows which indexes are in df_polyester; keeps NA in others
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# spandex from column df1[1]
df_spandex = df1.loc[df1[1].str.contains('spandex', na=True), 1]
df_spandex.name = 'spandex'
df_ref = pd.concat([df_ref, df_spandex], axis=1) # overwrites the rows which indexes are in df_spandex; keeps NA in others
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# column df1[2]
df1[2].unique()  # it shows we have only 'spandex'

# spandex from column df1[2]
df_spandex = df1.loc[df1[2].str.contains('spandex', na=True), 1]
df_spandex.name = 'spandex'
df_ref = pd.concat([df_ref, df_spandex], axis=1) # overwrites the rows which indexes are in df_spandex; keeps NA in others
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# final join
data = pd.concat([data, df_ref], axis=1)

#format composition data
data['cotton'] = data['cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['spandex'] = data['spandex'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
